In [1]:
%%bash
set -xeo pipefail

docker rm -f moto
docker run -d --rm -p 8000:5000 --name moto motoserver/moto:latest

cd ./aws_sns.infra
rm -rf .terraform.tfstate* # Clean up any previous state

+ docker rm -f moto


moto


+ docker run -d --rm -p 8000:5000 --name moto motoserver/moto:latest


bd1eb8bc0988e370fc75df2399988d830561be36afd392600b15c43c03139df2


+ cd ./aws_sns.infra
+ rm -rf '.terraform.tfstate*'


In [2]:
%%bash
set -xeo pipefail
cd aws_sns.infra

terraform init
terraform plan -out=.terraform/planlog

+ cd aws_sns.infra
+ terraform init



Initializing the backend...

Initializing provider plugins...
- Reusing previous version of hashicorp/aws from the dependency lock file
- Using previously-installed hashicorp/aws v5.31.0

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


+ terraform plan -out=.terraform/planlog


aws_sns_topic.sender: Refreshing state... [id=arn:aws:sns:us-east-1:123456789012:my-topic]
aws_sqs_queue.receiver: Refreshing state... [id=http://localhost:8000/123456789012/my-queue]
aws_sns_topic_subscription.receiver: Refreshing state... [id=arn:aws:sns:us-east-1:123456789012:my-topic:ea42bde3-f226-4f4e-942c-dd857f800c8e]

Note: Objects have changed outside of Terraform

Terraform detected the following changes made outside of Terraform since the
last "terraform apply" which may have affected this plan:

  # aws_sns_topic.sender has been deleted
  - resource "aws_sns_topic" "sender" {
      - arn                                      = "arn:aws:sns:us-east-1:123456789012:my-topic" -> null
        id                                       = "arn:aws:sns:us-east-1:123456789012:my-topic"
        name                                     = "my-topic"
        # (11 unchanged attributes hidden)
    }

  # aws_sqs_queue.receiver has been deleted
  - resource "aws_sqs_queue" "receiver" {
     

In [3]:
%%bash
set -xeo pipefail
cd aws_sns.infra
terraform apply -auto-approve .terraform/planlog

+ cd aws_sns.infra
+ terraform apply -auto-approve .terraform/planlog


aws_sqs_queue.receiver: Creating...
aws_sns_topic.sender: Creating...
aws_sns_topic.sender: Creation complete after 0s [id=arn:aws:sns:us-east-1:123456789012:my-topic]
aws_sqs_queue.receiver: Still creating... [10s elapsed]
aws_sqs_queue.receiver: Still creating... [20s elapsed]
aws_sqs_queue.receiver: Creation complete after 25s [id=http://localhost:8000/123456789012/my-queue]
aws_sns_topic_subscription.receiver: Creating...
aws_sns_topic_subscription.receiver: Creation complete after 0s [id=arn:aws:sns:us-east-1:123456789012:my-topic:e86550c4-082e-43d8-a0e8-cac99e5237ff]

Apply complete! Resources: 3 added, 0 changed, 0 destroyed.

Outputs:

sns_sender_arn = "arn:aws:sns:us-east-1:123456789012:my-topic"
sqs_receiver_url = "http://localhost:8000/123456789012/my-queue"


In [4]:
import os
import boto3

# https://github.com/getmoto/moto/issues/1941#issuecomment-557606526
os.environ["AWS_ACCESS_KEY_ID"] = "testing"
os.environ["AWS_SECRET_ACCESS_KEY"] = "testing"
os.environ["AWS_REGION"] = "us-east-1"

sns_sender_arn = "arn:aws:sns:us-east-1:123456789012:my-topic"
sqs_receiver_url = "http://localhost:8000/123456789012/my-queue"

sender_topic = boto3.resource(
    "sns",
    region_name="us-east-1",
    endpoint_url="http://localhost:8000",
).Topic(sns_sender_arn)

receiver_queue = boto3.resource(
    "sqs",
    region_name="us-east-1",
    endpoint_url="http://localhost:8000",
).Queue(sqs_receiver_url)

In [5]:
from dataclasses import dataclass
from typing import Self
from pyeventbus.aws.sns_event_bus import SNSEventBus
from pyeventbus.base.domain_event import DomainEvent
from pyeventbus.base.eventbus import EventBus

eventbus = SNSEventBus(sender_topic)

@dataclass
class UserCreatedEvent(DomainEvent):
    user_id: str
    
    def to_dict(self) -> dict[str, str]:
        return {"user_id": self.user_id}

    @classmethod
    def from_dict(cls, data: dict[str, str]) -> Self:
        return cls(data["user_id"])

@dataclass
class SendWelcomeEmailCommand:
    eventbus: EventBus
    def __post_init__(self):
        self.eventbus.subscribe(
            UserCreatedEvent,
            lambda event: self.send_welcome_email(event.user_id),
            self.__class__,
        )

    def send_welcome_email(self, user_id: str):
        print("[MOCKED] Sending email to user_id: ", user_id)

SendWelcomeEmailCommand(eventbus) # Register the handler

In [6]:
eventbus.publish(UserCreatedEvent("1234"))

In [7]:
import json
from mypy_boto3_sqs.service_resource import Message

messages: list[Message] = []
for message in receiver_queue.receive_messages():
    messages.append(message)
    message.delete()

assert len(messages) == 1, "Expected 1 message"
raw_sns_message = json.loads(messages[0].body)
raw_event = json.loads(raw_sns_message["Message"])

In [8]:
domain_event = eventbus.build_event_from_subscriptions(
    raw_event["event_type"], raw_event["event"]
)
domain_event

UserCreatedEvent(user_id='1234')

In [9]:
assert domain_event, "Expected a domain event"
for handler_cls in eventbus.get_handlers(type(domain_event)):
    eventbus.call_handler(domain_event, handler_cls)

[MOCKED] Sending email to user_id:  1234
